In [1]:
from alarms.utils import AnalysisArea, stringify_messages, define_message_type, define_region, timedelta_to_hours
import json
import pandas as pd
from datetime import datetime

In [2]:
SELECTED_REGION = AnalysisArea.LVIV

In [3]:
# Load Lviv data into dataframe
if SELECTED_REGION == AnalysisArea.LVIV:
    ALARM_START_MARKERS = [r"(?:\n|.)*Увага! Повітряна тривога(?:\n|.)*"]
    ALARM_END_MARKERS = [r"(?:\n|.)*Увага! Відбій повітряної тривоги(?:\n|.)*"]
    filename = "-1001399934598-channel-messages.json"
# Load all regions data into dataframe
if SELECTED_REGION == AnalysisArea.ALL_UKRAINE:
    ALARM_START_MARKERS = [r"(?:\n|.)*(?:Повітряна тривога|Загроза артобстрілу)(?: в|)(?:\n|.)*"]
    ALARM_END_MARKERS = [r"(?:\n|.)*Відбій (?:(?:повітряної |)тривоги|загрози артобстрілу)(?: в|)(?:\n|.)*"]
    filename = "-1766138888-channel-messages.json"
# Load data
with open(f"../exports/{filename}", "r") as f:  # NOQA
    raw_messages = json.loads(f.read())
raw_messages = pd.DataFrame(raw_messages["messages"]) if raw_messages.get("messages") else pd.DataFrame(
    raw_messages)
print(f"- Processing {len(raw_messages)} channel messages.")

- Processing 2365 channel messages.


In [4]:
# Convert raw messages into expected format
channel_messages = raw_messages[['date', 'text']].copy()
channel_messages.columns = ["datetime", "message"]
channel_messages["message"] = channel_messages["message"].apply(stringify_messages)
# Convert string into datetime
channel_messages["datetime"] = pd.to_datetime(channel_messages["datetime"], format="%Y-%m-%dT%H:%M:%S")
channel_messages.head()
# Skip dates < February 24th as alarms weren't working correctly during the first day of the war
channel_messages.drop(channel_messages[channel_messages["datetime"] < datetime(2022, 2, 25, 0, 0, 0, 0)].index, inplace=True)
# Skip dates > September 14th to keep data consistent
channel_messages.drop(channel_messages[channel_messages["datetime"] > datetime(2022, 9, 15, 0, 0, 0, 0)].index, inplace=True)
# Sort to get historical order of events
channel_messages = channel_messages.sort_values(by="datetime", ascending=True)
channel_messages

,datetime,message
1621,2022-02-25 06:53:08,Оповіщення ЦЗ:\nУвага! Повітряна тривога! Уваг...
1622,2022-02-25 07:56:00,Увага! Відбій повітряної тривоги!\nУвага! Відб...
1623,2022-02-25 07:56:16,Увага! Відбій повітряної тривоги!\nУвага! Відб...
1624,2022-02-25 12:17:40,Увага! Повітряна тривога! Увага! Повітряна три...
1625,2022-02-25 12:17:45,Увага! Повітряна тривога! Увага! Повітряна три...
...,...,...
2360,2022-09-11 20:44:41,Оповіщення ЦЗ:\nУвага! Відбій повітряної триво...
2361,2022-09-14 16:37:19,Увага! Повітряна тривога!\nУвага! Повітряна тр...
2362,2022-09-14 16:54:08,Оповіщення ЦЗ:\nУвага! Відбій повітряної триво...
2363,2022-09-14 16:59:10,Увага! Повітряна тривога!\nУвага! Повітряна тр...


In [5]:
# Mark info messages and actual alarms
channel_messages["message_type"] = channel_messages["message"].apply(  # NOQA
    define_message_type(ALARM_START_MARKERS, ALARM_END_MARKERS))  # NOQA
# # Look at alarm messages and copy the slice into separate datafame
alarm_messages_df = channel_messages[channel_messages["message_type"] != "info"].copy()
print(f"- Collected {len(alarm_messages_df)} alarm messages.")
alarm_messages_df

- Collected 689 alarm messages.


,datetime,message,message_type
1621,2022-02-25 06:53:08,Оповіщення ЦЗ:\nУвага! Повітряна тривога! Уваг...,start
1622,2022-02-25 07:56:00,Увага! Відбій повітряної тривоги!\nУвага! Відб...,end
1623,2022-02-25 07:56:16,Увага! Відбій повітряної тривоги!\nУвага! Відб...,end
1624,2022-02-25 12:17:40,Увага! Повітряна тривога! Увага! Повітряна три...,start
1625,2022-02-25 12:17:45,Увага! Повітряна тривога! Увага! Повітряна три...,start
...,...,...,...
2360,2022-09-11 20:44:41,Оповіщення ЦЗ:\nУвага! Відбій повітряної триво...,end
2361,2022-09-14 16:37:19,Увага! Повітряна тривога!\nУвага! Повітряна тр...,start
2362,2022-09-14 16:54:08,Оповіщення ЦЗ:\nУвага! Відбій повітряної триво...,end
2363,2022-09-14 16:59:10,Увага! Повітряна тривога!\nУвага! Повітряна тр...,start


In [6]:
# Look at info messages
info_messages_df = channel_messages[channel_messages["message_type"] == "info"]
print(f"- Collected {len(info_messages_df)} info messages.")
info_messages_df

- Collected 55 info messages.


,datetime,message,message_type
1628,2022-02-25 14:44:58,Важлива інформація:\nhttps://www.facebook.com/...,info
1629,2022-02-25 18:41:11,Шановні мешканці Львівщини!\nВже два дні ми за...,info
1630,2022-02-25 18:59:25,Комендантська година: що і як:,info
1631,2022-02-25 18:59:36,,info
1632,2022-02-25 20:38:59,https://www.facebook.com/100064486998276/posts...,info
1633,2022-02-25 20:46:43,​​Інформація про системи оповіщення щодо комен...,info
1638,2022-02-26 08:58:47,Львів’ян попереджають про заборону польотів лі...,info
1648,2022-02-27 11:11:15,До уваги голів сільських та селищних територіа...,info
1649,2022-02-27 11:42:48,Фото від *@@##₴₴*,info
1650,2022-02-27 11:43:06,Офіційні канали комунікації Львівської обласно...,info


In [7]:
# Define region per alarm
if SELECTED_REGION == AnalysisArea.LVIV:
    alarm_messages_df["region"] = "Lviv region"
if SELECTED_REGION == AnalysisArea.ALL_UKRAINE:
    alarm_messages_df["region"] = alarm_messages_df["message"].apply(define_region)
    # Drop test region
    alarm_messages_df.drop(alarm_messages_df[alarm_messages_df["region"] == "Тестовий Регіон"].index, inplace=True)
    # Focus only on regions, drop smaller areas (cities, districts, etc.)
    alarm_messages_df.drop(alarm_messages_df[~alarm_messages_df["region"].str.contains("region")].index, inplace=True)

In [8]:
if SELECTED_REGION == AnalysisArea.LVIV:
    # Filter all the duplicates, as Lviv alarms are usually (but not always) annouced 2-3 times in a row
    clean_alarm_messages = []
    for _, alarm_row in alarm_messages_df.iterrows():
        # The first entry is always a start
        if len(clean_alarm_messages) == 0:
            clean_alarm_messages.append(alarm_row.to_dict())
            continue
        # Skip duplicates (if the previous message have the same type)
        if clean_alarm_messages[-1]["message_type"] == alarm_row["message_type"]:
            continue
        clean_alarm_messages.append(alarm_row.to_dict())
if SELECTED_REGION == AnalysisArea.ALL_UKRAINE:
    # Filtering the first bugged message
    clean_alarm_messages = [alarm_row.to_dict() for _, alarm_row in alarm_messages_df.iterrows()][1:]
print(f"- Collected {len(clean_alarm_messages)} clean alarm messages.")  # NOQA

- Collected 390 clean alarm messages.


In [9]:
# Combine alarm messages into actual alarms
alarms = []
alarms_without_start = []
for alarm_message in clean_alarm_messages:
    # Create alarms from start messages
    if alarm_message["message_type"] == "start":
        alarms.append({"start_datetime": alarm_message["datetime"], "start_message": alarm_message["message"],
                       "region": alarm_message["region"]})
        continue
    # Find start message index, when iterating end messages
    smi = None
    for i in range(len(alarms) - 1, -1, -1):
        if alarms[i]["region"] == alarm_message["region"]:
            smi = i
            break
    # Skip alarms with no start message (bugs)
    if smi is None:
        alarms_without_start.append(alarm_message)
        continue
    # Extend the start message with end message
    alarms[smi]["end_datetime"] = alarm_message["datetime"]
    alarms[smi]["end_message"] = alarm_message["message"]
    alarms[smi]["timedelta"] = alarms[smi]["end_datetime"] - alarms[smi]["start_datetime"]
    alarms[smi]["duration_hours"] = timedelta_to_hours(alarms[smi]["timedelta"])
print(f"- Can't find start messages for {len(alarm_message)} alarms, skipping.")  # NOQA
all_alarms_df = pd.DataFrame(alarms)
all_alarms_df["day_of_the_year"] = all_alarms_df["start_datetime"].apply(lambda x: x.timetuple().tm_yday)
print(f"- Collected {len(all_alarms_df)} alarms.")
all_alarms_df.head(7)

- Can't find start messages for 4 alarms, skipping.
- Collected 195 alarms.


,start_datetime,start_message,region,end_datetime,end_message,timedelta,duration_hours,day_of_the_year
0,2022-02-25 06:53:08,Оповіщення ЦЗ:\nУвага! Повітряна тривога! Уваг...,Lviv region,2022-02-25 07:56:00,Увага! Відбій повітряної тривоги!\nУвага! Відб...,0 days 01:02:52,1.047778,56
1,2022-02-25 12:17:40,Увага! Повітряна тривога! Увага! Повітряна три...,Lviv region,2022-02-25 13:00:34,Увага! Відбій повітряної тривоги!\nУвага! Відб...,0 days 00:42:54,0.715000,56
2,2022-02-26 06:15:28,Увага! Повітряна тривога! Увага! Повітряна три...,Lviv region,2022-02-26 07:11:56,Увага! Відбій повітряної тривоги!\nУвага! Відб...,0 days 00:56:28,0.941111,57
3,2022-02-26 13:46:10,🚨Увага! Повітряна тривога❗️\nУвага! Повітряна ...,Lviv region,2022-02-26 14:46:45,❗️Увага! Відбій повітряної тривоги❗️\n🇬🇧Warnin...,0 days 01:00:35,1.009722,57
4,2022-02-26 17:18:39,🚨Увага! Повітряна тривога❗️\nУвага! Повітряна ...,Lviv region,2022-02-26 18:06:54,❗️Увага! Відбій повітряної тривоги❗️\n🇬🇧Warnin...,0 days 00:48:15,0.804167,57
5,2022-02-26 19:59:11,🚨Увага! Повітряна тривога❗️\nУвага! Повітряна ...,Lviv region,2022-02-26 20:42:10,❗️Увага! Відбій повітряної тривоги❗️\n🇬🇧Warnin...,0 days 00:42:59,0.716389,57
6,2022-02-28 20:54:13,Увага! Повітряна тривога! Увага! Повітряна три...,Lviv region,2022-02-28 21:23:49,Увага! Відбій повітряної тривоги! Увага! Відбі...,0 days 00:29:36,0.493333,59


In [10]:
# Save to file to create map
if SELECTED_REGION == AnalysisArea.ALL_UKRAINE:
    filename = "ukraine_alarms.json"
if SELECTED_REGION == AnalysisArea.LVIV:
    filename = "lviv_alarms.json"
with open(f"processed_data/{filename}", "w") as f:  # NOQA
    f.write(all_alarms_df.to_json())